In [ ]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input,Dense,Dropout,Activation,Flatten,GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Lambda,Concatenate
from tensorflow.keras import optimizers, regularizers, applications
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model

import numpy as np
import scipy.io as sio
import scipy.stats as sst
import scipy.special as ssp
import scipy

import random
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve, roc_auc_score, roc_curve,precision_recall_curve,average_precision_score

import matplotlib.pyplot as plt

import keras

In [ ]:
plt.rcParams["font.family"] = "Linux Biolinum O"
plt.rcParams.update({'font.size': 18, 'font.weight': 'bold'})
plt.rcParams["legend.handlelength"] = 1.0

In [ ]:
## load data
data = sio.loadmat('./D1_test_features.mat')

In [ ]:
logits = data['logits']
prediction = data['prediction'].reshape([-1])
label = data['label'].reshape([-1])
probability = data['probability']
TF = prediction!=label

In [ ]:
1-np.mean(TF)

In [ ]:
## baseline on temperature scaling
# sample 1% data to obtain T
sample_N = logits.shape[0];
np.random.seed(seed=42)
sample_index = np.random.choice(sample_N,int(sample_N*0.01),replace=False)

sample_logits = logits[sample_index];print(sample_logits.shape)
sample_TF = TF[sample_index];print(sample_TF.shape)
sample_label = label[sample_index];print(sample_label.shape)

T_arr = np.arange(0.2,10,0.1)
NLL_train = []
aupr_train = []
for T in T_arr:
    #NLL
    cur_prob = ssp.softmax(sample_logits/T, axis=-1)
    aupr_train.append(average_precision_score(sample_TF,1-np.max(cur_prob,axis=-1)))
    NLL_train.append(np.nanmean(sst.entropy(to_categorical(sample_label,43),cur_prob)))

In [ ]:
plt.figure(figsize=(5,3))
plt.plot(T_arr,NLL_train)
plt.xlim([0,10]);
xindex=NLL_train.index(min(NLL_train))
plt.plot(T_arr[xindex],NLL_train[xindex],'r*')
print(T_arr[xindex])
plt.grid();plt.xlabel('Tempearture (T)');plt.ylabel('NLL');

In [ ]:
## original data baseline max(probability)
calibrated_probability = ssp.softmax(logits/2.9, axis=-1)
print(roc_auc_score(TF,1-np.max(calibrated_probability,axis=-1)),
      average_precision_score(TF,1-np.max(calibrated_probability,axis=-1)))

In [ ]:
np.mean(np.max(calibrated_probability,axis=-1))

In [ ]:
# baseline
np.mean(np.max(probability,axis=-1))

In [ ]:
# data used for transfer
## load encode feature, prob and tcp
monitor_test_X = probability
monitor_test_Y = TF
max_P = np.max(monitor_test_X,axis=-1)
max_P = (max_P-min(max_P))/(max(max_P)-min(max_P))
print(monitor_test_X.shape, monitor_test_Y.shape,max_P.shape)

In [ ]:
## simulation RS 10k, 100,1000
accuracy_list = []
for i in range(100):
    cur_accuracy = 1-np.mean(monitor_test_Y[np.random.choice(monitor_test_Y.shape[0],101*10)])
    accuracy_list.append(cur_accuracy)

In [ ]:
np.min(accuracy_list),np.max(accuracy_list)

# according to sampled 1%, decide the MP-based and Entropy-based

In [ ]:
## baseline on temperature scaling
# sample 1% data to obtain T
sample_N = logits.shape[0];
np.random.seed(seed=42)
sample_index = np.random.choice(sample_N,int(sample_N*0.01),replace=False)

sample_logits = logits[sample_index];print(sample_logits.shape)
sample_TF = TF[sample_index];print(sample_TF.shape)
sample_label = label[sample_index];print(sample_label.shape)
sample_probability = probability[sample_index];print(sample_probability.shape)

In [ ]:
max_P = np.max(sample_probability,axis=-1)

In [ ]:
my_thre = np.arange(0.8,1,0.005)
res = []
for thre in my_thre:
    res.append(np.mean(max_P>thre))

In [ ]:
1-np.mean(sample_TF)

In [ ]:
plt.figure(figsize=(5,3))
plt.plot(my_thre,res,'r',markevery=2);
plt.plot(0.95,0.83,'b*')
plt.xlim([0.8,1]);plt.xlabel('threshold');plt.ylabel('Estimated accuracy');

In [ ]:
# MP is 0.95
MP_th = 0.95
temp_score = np.max(probability,axis=-1)>MP_th
np.mean(temp_score)

In [ ]:
## entropy based follows

In [ ]:
## calcuate entropy
test_entropy = np.array([sst.entropy(i) for i in probability]);print(test_entropy.shape)
MAX_ENTROPY = sst.entropy(np.ones(43,)/43);print(MAX_ENTROPY)
test_entropy = test_entropy/MAX_ENTROPY#np.max(test_entropy)

In [ ]:
sample_entropy = test_entropy[sample_index];print(sample_entropy.shape)

In [ ]:
my_thre = np.arange(0,0.1,0.001)
res = []
for thre in my_thre:
    res.append(np.mean(sample_entropy<thre))

In [ ]:
1-np.mean(sample_TF)

In [ ]:
plt.figure(figsize=(5,3))
plt.plot(my_thre,res,'r',markevery=2);
plt.plot(0.05,0.82,'b*')# plt.xlim([0.8,1]);
plt.xlabel('threshold');plt.ylabel('Estimated accuracy');

In [ ]:
# M tnetropy is 0.05
MP_th = 0.05
temp_score = test_entropy<MP_th
np.mean(temp_score)

In [ ]:
## get auroc and aupr
temp_score = test_entropy<MP_th
roc_auc_score(TF,1-temp_score), average_precision_score(TF,1-temp_score)

In [ ]:
#---------------------

In [ ]:
# MP is 0.95
MP_th = 0.95
temp_score = np.max(probability,axis=-1)>MP_th
np.mean(temp_score)

In [ ]:
## get auroc and aupr
temp_score = np.max(probability,axis=-1)>MP_th
roc_auc_score(TF,1-temp_score), average_precision_score(TF,1-temp_score)

In [ ]:
# for max(P)

In [ ]:
## get auroc and aupr
temp_score = np.max(probability,axis=-1)
roc_auc_score(TF,1-temp_score), average_precision_score(TF,1-temp_score)

In [ ]:
## get auroc and aupr
temp_score = 1-test_entropy
roc_auc_score(TF,1-temp_score), average_precision_score(TF,1-temp_score)